In [ ]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, WeightedRandomSampler
from PIL import Image
from pycocotools.coco import COCO
import albumentations as A

In [ ]:
class FashionDataset(Dataset):
    def __init__(self, root, ann_file, feature_extractor, train=False, num_attributes=294):
        """
        Args:
            root (str): Đường dẫn thư mục ảnh.
            ann_file (str): Đường dẫn file JSON annotation.
            feature_extractor: YolosImageProcessor từ HuggingFace.
            train (bool): Nếu True sẽ áp dụng Augmentation.
        """
        self.root = root
        self.coco = COCO(os.path.join(root, ann_file))
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.feature_extractor = feature_extractor
        self.num_attributes = num_attributes
        self.train = train
        
        # Khởi tạo Augmentation pipeline
        self.transforms = self.get_transforms(train=train)

    def get_transforms(self, train=False):
        """
        Cấu hình Augmentation - Chìa khóa xử lý Small Objects & Imbalance
        """
        if train:
            return A.Compose([
                # --- CHIẾN LƯỢC CHO VẬT THỂ NHỎ (SMALL OBJECTS) ---
                # RandomCrop giúp model "nhìn gần" hơn vào chi tiết (nhẫn, đồng hồ)
                # Thay vì resize ảnh 3000px xuống 800px (mất chi tiết), ta cắt lấy vùng 800x800
                A.OneOf([
                    A.RandomCrop(width=800, height=800, p=0.5),
                    A.RandomResizedCrop(width=800, height=800, scale=(0.5, 1.0), p=0.5),
                ], p=0.6),

                # --- CHIẾN LƯỢC TĂNG ĐA DẠNG DỮ LIỆU ---
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.3),
                
                # Biến đổi màu sắc để model không phụ thuộc vào ánh sáng
                A.RandomBrightnessContrast(p=0.2),
                A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.2),
                
                # Đảm bảo ảnh đầu ra không quá lớn gây tràn RAM
                A.LongestMaxSize(max_size=1333), 
                A.PadIfNeeded(min_height=800, min_width=800, border_mode=cv2.BORDER_CONSTANT, value=[124, 116, 104])
            ], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids', 'attribute_ids'], min_visibility=0.3))
        else:
            # Với tập Val/Test, chỉ Resize cơ bản
            return A.Compose([
                A.LongestMaxSize(max_size=1333),
                A.PadIfNeeded(min_height=800, min_width=800, border_mode=cv2.BORDER_CONSTANT, value=[124, 116, 104])
            ], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids', 'attribute_ids']))

    def __getitem__(self, index):
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        coco_target = coco.loadAnns(ann_ids)

        # 1. Load ảnh
        path = coco.loadImgs(img_id)[0]['file_name']
        img_path = os.path.join(self.root, path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Albumentations dùng numpy RGB

        # 2. Parse Annotations ban đầu
        boxes = []
        category_ids = []
        attribute_ids_list = [] # List các list attributes
        area = []
        iscrowd = []

        for ann in coco_target:
            x, y, w, h = ann['bbox']
            if w < 1 or h < 1: continue
            
            boxes.append([x, y, w, h]) # COCO format cho Albumentations
            category_ids.append(ann['category_id'])
            area.append(ann['area'])
            iscrowd.append(ann['iscrowd'])
            
            # Giữ lại attribute_ids để transform theo box
            attrs = ann.get('attribute_ids', [])
            attribute_ids_list.append(attrs)

        # 3. Áp dụng Augmentation (Crop, Flip, etc.)
        if self.transforms:
            try:
                transformed = self.transforms(
                    image=image, 
                    bboxes=boxes, 
                    category_ids=category_ids,
                    attribute_ids=attribute_ids_list
                )
                image = transformed['image']
                boxes = transformed['bboxes']
                category_ids = transformed['category_ids']
                attribute_ids_list = transformed['attribute_ids']
            except ValueError:
                # Fallback nếu augmentation lỗi (thường do box nằm ngoài ảnh sau crop)
                pass

        # 4. Format lại dữ liệu cho YOLOS Processor
        # YOLOS yêu cầu boxes dạng [x_min, y_min, x_max, y_max]
        final_boxes = []
        final_attributes = []
        
        for i, box in enumerate(boxes):
            x, y, w, h = box
            final_boxes.append([x, y, x + w, y + h])
            
            # Xử lý Multi-hot vector cho Attributes
            attr_vec = torch.zeros(self.num_attributes, dtype=torch.float32)
            valid_ids = [aid for aid in attribute_ids_list[i] if aid < self.num_attributes]
            if valid_ids:
                attr_vec[valid_ids] = 1.0
            final_attributes.append(attr_vec)

        # 5. Đóng gói Target
        target = {}
        target["boxes"] = torch.as_tensor(final_boxes, dtype=torch.float32)
        target["class_labels"] = torch.as_tensor(category_ids, dtype=torch.long)
        target["image_id"] = torch.tensor([img_id])
        
        # Xử lý trường hợp ảnh không còn box nào sau khi Crop
        if len(final_attributes) > 0:
            target["attribute_labels"] = torch.stack(final_attributes)
            target["area"] = torch.as_tensor(area[:len(final_boxes)], dtype=torch.float32) # Area có thể sai lệch sau crop nhưng tạm chấp nhận
            target["iscrowd"] = torch.as_tensor(iscrowd[:len(final_boxes)], dtype=torch.int64)
        else:
             # Tạo tensor rỗng nếu mất hết object
            target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
            target["class_labels"] = torch.zeros((0,), dtype=torch.long)
            target["attribute_labels"] = torch.zeros((0, self.num_attributes), dtype=torch.float32)
            target["area"] = torch.zeros((0,), dtype=torch.float32)
            target["iscrowd"] = torch.zeros((0,), dtype=torch.int64)

        # 6. Feature Extractor (Normalization & Formatting cuois cùng)
        # Lưu ý: Ta truyền ảnh đã augment (numpy) vào, processor sẽ convert sang Tensor
        encoding = self.feature_extractor(
            images=image, 
            annotations=target, 
            return_tensors="pt"
        )
        
        pixel_values = encoding["pixel_values"].squeeze()
        target = encoding["labels"][0]

        return pixel_values, target

    def __len__(self):
        return len(self.ids)

    def get_weighted_sampler(self):
        """
        Tính toán trọng số lấy mẫu (Sampling Weights) để xử lý Imbalanced Data.
        Logic: Ảnh nào chứa class hiếm (VD: Nhẫn) sẽ có trọng số cao hơn.
        """
        print("Đang tính toán Class Weights để cân bằng dữ liệu...")
        
        # 1. Đếm tần suất xuất hiện của từng Class
        class_counts = {}
        img_class_map = {} # img_id -> list of classes
        
        for img_id in self.ids:
            ann_ids = self.coco.getAnnIds(imgIds=img_id)
            anns = self.coco.loadAnns(ann_ids)
            classes = [ann['category_id'] for ann in anns]
            img_class_map[img_id] = classes
            
            for c in classes:
                class_counts[c] = class_counts.get(c, 0) + 1
        
        # 2. Tính Weight cho từng Class (Inverse Frequency)
        # Weight = Tổng số mẫu / Số mẫu của class đó
        total_samples = sum(class_counts.values())
        class_weights = {c: total_samples / (cnt + 1e-6) for c, cnt in class_counts.items()}
        
        # 3. Gán Weight cho từng Ảnh
        # Weight của ảnh = Weight lớn nhất của class có trong ảnh đó
        # (Ví dụ: Ảnh có Áo (common) và Nhẫn (rare) -> Lấy weight của Nhẫn)
        sample_weights = []
        for img_id in self.ids:
            classes = img_class_map.get(img_id, [])
            if not classes:
                weight = 0.0 # Bỏ qua ảnh không có object
            else:
                # Lấy max weight (ưu tiên class hiếm nhất trong ảnh)
                weight = max([class_weights.get(c, 0) for c in classes])
            sample_weights.append(weight)
            
        sample_weights = torch.as_tensor(sample_weights, dtype=torch.double)
        
        # Trả về Sampler
        sampler = WeightedRandomSampler(
            weights=sample_weights,
            num_samples=len(sample_weights),
            replacement=True
        )
        return sampler